In [35]:
import json
import numpy as np
import pandas as pd
import planetary_computer as pc
import pystac_client
from odc.stac import stac_load
from os import path
from tqdm import tqdm

In [36]:
with open('keys/planetary_computer_key.json') as key:
	key = json.load(key)['API_KEY']
	pc.settings.set_subscription_key(key)

In [37]:
crop_presence_data = pd.read_csv("./data/Crop_Location_Data_20221201.csv")
crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


In [38]:
def get_sentinel_data(latlong, time_slice, assets):
    '''
    Returns VV and VH values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    assets - A list of bands to be extracted
    '''

    box_size_deg = 0.0004 # Surrounding box in degrees, yields approximately 5x5 pixel region

    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')

    min_lon = float(latlong[1])-box_size_deg/2
    min_lat = float(latlong[0])-box_size_deg/2
    max_lon = float(latlong[1])+box_size_deg/2
    max_lat = float(latlong[0])+box_size_deg/2

    bbox_of_interest = (min_lon, min_lat, max_lon, max_lat)
    time_of_interest = time_slice

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    items = list(search.get_all_items())
    bands_of_interest = assets
    data = stac_load([items[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    vh = data["vh"].astype("float").values.tolist()[0][0]
    vv = data["vv"].astype("float").values.tolist()[0][0]
    
    return vh,vv

In [39]:
data_path = './data/planetary_computer/vh_vv_data_bbox5x5.json'

if not path.exists(data_path):
    time_slice = "2020-01-01/2020-12-31"
    assets = ['vh','vv']
    vh_vv = []

    for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
        vh_vv.append(get_sentinel_data(coordinates, time_slice, assets))
    vh_vv_data = pd.DataFrame(vh_vv, columns =['vh', 'vv'])

    vh_vv_data.to_json(data_path, indent=4)
else: vh_vv_data = pd.read_json(data_path)

100%|██████████| 600/600 [05:38<00:00,  1.77it/s]


In [40]:
vh_vv_data.head()

,vh,vv
0,0.007940,0.229033
1,0.001958,0.026450
2,0.014930,0.176809
3,0.008318,0.067144
4,0.009465,0.199129


In [41]:
test_file = pd.read_csv('data/submission_template.csv')
test_file.head()

,id,target
0,"(10.18019073690894, 105.32022315786804)",NaN
1,"(10.561107033461816, 105.12772097986661)",NaN
2,"(10.623790611954897, 105.13771401411867)",NaN
3,"(10.583364246115156, 105.23946127195805)",NaN
4,"(10.20744446668854, 105.26844107128906)",NaN


In [43]:
data_path = 'data/planetary_computer/submission_vh_vv_data_bbox5x5.json'

if not path.exists(data_path):
    time_slice = "2020-01-01/2020-12-31"
    assests = ['vh','vv']
    vh_vv = []

    for coordinates in tqdm(test_file['id']):
        vh_vv.append(get_sentinel_data(coordinates, time_slice,assests))
    submission_vh_vv_data = pd.DataFrame(vh_vv,columns=['vh','vv'])

    submission_vh_vv_data.to_json(data_path, indent=4)
else: submission_vh_vv_data = pd.read_json(data_path)

100%|██████████| 250/250 [04:32<00:00,  1.09s/it]


In [44]:
def calculate_RVI(vh_vv_data):
    rvi = []
    for i in range(vh_vv_data.shape[0]):
        vh = vh_vv_data['vh'][i]
        vv = vh_vv_data['vv'][i]
        rvi.append(4*np.sqrt(1-vv/(vv+vh))*(vh/(vv+vh)))

    return pd.DataFrame(rvi, columns=['RVI'])

In [45]:
rvi = calculate_RVI(vh_vv_data)
rvi_sub = calculate_RVI(submission_vh_vv_data)

In [46]:
rvi.head()

,RVI
0,0.024531
1,0.072372
2,0.086913
3,0.146389
4,0.038663


In [47]:
rvi_sub.head()

,RVI
0,0.005950
1,0.014730
2,0.205469
3,0.980703
4,0.151326


In [48]:
def combine_three_datasets(dataset1, dataset2, dataset3):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    dataset3 - Dataset 3 to be combined
    '''
    data = pd.concat([dataset1, dataset2, dataset3], axis=1)
    return data

In [49]:
crop_data = combine_three_datasets(crop_presence_data, vh_vv_data, rvi)

crop_data.head()

,Latitude and Longitude,Class of Land,vh,vv,RVI
0,"(10.323727047081501, 105.2516346045924)",Rice,0.007940,0.229033,0.024531
1,"(10.322364360592521, 105.27843410554115)",Rice,0.001958,0.026450,0.072372
2,"(10.321455902933202, 105.25254306225168)",Rice,0.014930,0.176809,0.086913
3,"(10.324181275911162, 105.25118037576274)",Rice,0.008318,0.067144,0.146389
4,"(10.324635504740822, 105.27389181724476)",Rice,0.009465,0.199129,0.038663


In [53]:
validation_data = combine_three_datasets(test_file, submission_vh_vv_data, rvi_sub)
validation_data.drop(columns=['target'], inplace=True)
validation_data.columns = ['Latitude and Longitude', 'vh', 'vv', 'RVI']

validation_data.head()

,Latitude and Longitude,vh,vv,RVI
0,"(10.18019073690894, 105.32022315786804)",0.005058,0.383090,0.005950
1,"(10.561107033461816, 105.12772097986661)",0.001983,0.081168,0.014730
2,"(10.623790611954897, 105.13771401411867)",0.017813,0.111093,0.205469
3,"(10.583364246115156, 105.23946127195805)",0.003966,0.006158,0.980703
4,"(10.20744446668854, 105.26844107128906)",0.020260,0.159520,0.151326


In [ ]:
crop_data.to_csv('./data/crop_data_bbox5x5.csv', index=False)
validation_data.to_csv('./data/validation_data_bbox5x5.csv', index=False)